<a href="https://colab.research.google.com/github/maksmanikowski/neural_networks/blob/main/dl04_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcinsawinski/UEP_KIE_DL_CODE/blob/main/dl03_training_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Exercise 4

Build churn prediction model usin data from:
https://www.kaggle.com/datasets/blastchar/telco-customer-churn/

copy of data avialable: data/churn.csv


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv")

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
y = df.iloc[:, -1]
X = df.iloc[:, :-1]

In [ ]:
X.iloc[:3]

In [ ]:
X.drop("customerID", axis=1, inplace=True)

In [ ]:
X.info()

In [ ]:
X["TotalCharges"] = pd.to_numeric(X["TotalCharges"], errors="coerce")

In [ ]:
X["TotalCharges"].info()

In [ ]:
nans = X[X["TotalCharges"].isnull()].index.tolist()
y.drop(nans, axis=0, inplace=True)
X.dropna(inplace=True)

In [ ]:
X = pd.get_dummies(X)
X.head()

In [ ]:
y = y.map({'Yes': 1, 'No': 0})
y[:5]

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
y_test.value_counts()

In [ ]:
y_train.value_counts()

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(45, input_shape=(45,), activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

opt = tf.keras.optimizers.Adamax(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=[tf.keras.metrics.AUC(name='auc')])

In [ ]:
import os, datetime
logdir = os.path.join("logs", "telco", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights= True)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    batch_size=128,
    epochs=50,
    verbose=1,
    callbacks=[early_stopping, tensorboard_callback]
)

In [ ]:
model.summary()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

- Why loss is higher in training data?